In [12]:
import pandas as pd
import os
import numpy as np

import Utils as ut
from sklearn.svm import SVR
from sklearn.preprocessing import normalize
 
from sklearn.model_selection import StratifiedShuffleSplit, cross_val_score, cross_validate

In [13]:
d_clinical = pd.read_csv(os.path.join('data', 'dataset_clinical_cleaned.csv'))
#d_genetic = pd.read_csv(os.path.join('data', 'dataset_genetic_cleaned_noOHE.csv'))
#d_vampire = pd.read_csv(os.path.join('data', 'dataset_vampire_cleaned.csv'))
outputs = pd.read_csv(os.path.join('data', 'outputs_cleaned.csv'))

In [14]:
y_class = outputs["dement_fail"].values
meaningful_idxs = np.where(y_class==1)
y = outputs["dement_time_age"].values[meaningful_idxs]
C = d_clinical.values[meaningful_idxs]
#G = d_genetic.values[meaningful_idxs]
#V = d_vampire.values[meaningful_idxs]

## Dementia

In [15]:
y = outputs["dement_fail"].values
C = d_clinical.values
#G = d_genetic.values
#V = d_vampire.values

In [16]:
# COMPUTATIONAL COMPLEXITY: Reduce #samples
tr_idx, ts_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=0.25).split(C, y))

In [17]:
y_ = y[tr_idx]
y_test = y[ts_idx]
C_ = C[tr_idx]
C_test = C[ts_idx]
#G_ = G[tr_idx]
#G_test = G[ts_idx]
#V_ = V[tr_idx]
#V_test = V[ts_idx]

In [18]:
ds_list = [C_]#, G_, V_]
ds_test = [C_test]#, G_test, V_test]
ds_names = ['clinic']#, 'genetic', 'vampire']

In [19]:
def centering_normalizing(train, test, exclusion_list = None):
    
    if exclusion_list is not None:
        scale, train = ut.centering_normalizing(train, exclusion_list)
    else:
        scale, train = ut.centering_normalizing(train)

    new_Xts = test-scale

    if exclusion_list is not None:
        new_Xts[:, exclusion_list] = test[:, exclusion_list]

    return train, new_Xts, scale

def normalizing(train, test):
    return normalize(train), normalize(test)

In [20]:
def make_regression(X, y, reg):

    scores = np.zeros(len(reg))
    
    for idx, r in enumerate(reg):
        model = SVR(C = r)
        scores[idx] = np.mean(cross_validate(model,  X, y, return_train_score=False, cv=3)['test_score'])
        
    return scores[np.argmax(scores)]

In [21]:
def learn(C_, C_test, y, y_test, sparsity, centering = False, norm = False):
    if centering == True:
        C_, C_test, _ = centering_normalizing(C_, C_test)#, [5,6,7,9,10,13,15,16,17,18,19])

    if norm == True:
         C_, C_test = normalizing(C_, C_test)

    best_lambda = make_regression(C_, y, sparsity)
    model = SVR(C = best_lambda)
    model.fit(C_, y)
    y_pred = model.predict(C_test)
    #if kernel == 'linear': coef = model.coef_
    
    meanErr = np.mean(np.abs(y_pred-y_test))
    varErr = np.var(np.abs(y_pred-y_test))

    print("Average error: {}".format(meanErr))
    print("Error variance: {}".format(varErr))
    #if kenel == 'linear': print("Coef: {}".format(coef))
    print("BestLambda: {}".format(best_lambda))

In [ ]:
lamb = [0.01, 0.03, 0.05, 0.07, 0.1, 0.12]

## Basic approach

In [23]:
learn(C_, C_test, y_, y_test, lamb)

ValueError: C <= 0

## Normalized data

In [22]:
learn(C_, C_test, y_, y_test, lamb, norm = True)

ValueError: C <= 0

## Origin Data Centering

In [ ]:
learn(C_, C_test, y_, y_test, lamb, centering = True)

## Origin Data  Centering and Normalization

In [ ]:
learn(C_, C_test, y_, y_test, lamb, centering = True, norm = True)